# Set Up

In [ ]:
from IPython.display import clear_output 
!pip install gcsfs
!pip install t5==0.9.2
!pip install -q tensorflow-text==2.8.0rc0
clear_output()

In [ ]:
print("Installing dependencies...")
import functools
import os
import gin
import tensorflow_gcs_config
from google.colab import auth
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
from contextlib import contextmanager
import logging as py_logging
import t5

In [ ]:
print("Setting up GCS access...")
os.environ['USE_AUTH_EPHEM'] = '0'
from google.colab import auth
auth.authenticate_user()

# Set credentials for GCS reading/writing from Colab and TPU.
TPU_TOPOLOGY = "2x2"
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  TPU_ADDRESS = tpu.get_master()
  print('Running on TPU:', TPU_ADDRESS)
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
tf.config.experimental_connect_to_host(TPU_ADDRESS)
tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()


#LOGGING
tf.get_logger().propagate = False
py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

# Load Vocabulary

In [ ]:
TOKENIZER_DIR = "gs://cs-summarizer/linking&classification" #@param { type: "string" }
if not TOKENIZER_DIR or TOKENIZER_DIR == "gs://": 
  raise ValueError("You must enter a TOKENIZER_DIR.")

VOCAB_PREFIX = 'tokenizer/tokenizer-linking' #@param {type: "string"}
vocab_model_path = os.path.join(TOKENIZER_DIR, f'{VOCAB_PREFIX}.model')
vocab_path = os.path.join(TOKENIZER_DIR, f'{VOCAB_PREFIX}.vocab')
print(vocab_model_path)
print(vocab_path)

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary

num_special_mask_tokens = 100 #@param {type: "integer"}

def load_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, num_special_mask_tokens)

# Prepare Dataset for T5

In [ ]:
train_path_linking = 'gs://cs-summarizer/linking&classification/datasets/linking/train.tsv' #@param { type: "string" }
eval_path_linking = 'gs://cs-summarizer/linking&classification/datasets/linking/eval.tsv' #@param { type: "string" }
test_path_linking = 'gs://cs-summarizer/linking&classification/datasets/linking/test.tsv' #@param { type: "string" }
finetune_datasets_paths_linking = {
    "train":      train_path_linking,
    "validation": test_path_linking
}

train_path_classification = 'gs://cs-summarizer/linking&classification/datasets/classification/train.tsv' #@param { type: "string" }
eval_path_classification = 'gs://cs-summarizer/linking&classification/datasets/classification/eval.tsv' #@param { type: "string" }
test_path_classification = 'gs://cs-summarizer/linking&classification/datasets/classification/test.tsv' #@param { type: "string" }
finetune_datasets_paths_classification = {
    "train":      train_path_classification,
    "validation": test_path_classification
}


num_input_examples_linking = dict(train=3992, validation=704)  
num_input_examples_classification = dict(train=9250, validation=1157) 

In [ ]:
def load_dataset_linking(split, shuffle_files=True):
  """
  Function to load .tsv dataset as a tf.data.Dataset in TensorFlow
  """
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(finetune_datasets_paths_linking[split])
  ds = ds.map(functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                          field_delim="\t", use_quote_delim=True)
                          , 
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

def load_dataset_classification(split, shuffle_files=True):
  """
  Function to load .tsv dataset as a tf.data.Dataset in TensorFlow
  """
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(finetune_datasets_paths_classification[split])
  ds = ds.map(functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                          field_delim="\t", use_quote_delim=True)
                          , 
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

### A few examples

In [ ]:
print("A few raw validation examples...")
for ex in tfds.as_numpy(load_dataset_classification("validation").take(5)):
  print(ex)

# Dataset Prepocessing 

In [ ]:
from tensorflow_datasets.core.utils.type_utils import Shape
def preprocessing_linking(ds):
  """
  Preprocess function to convert the tf.data.Dataset into a text-to-text format,
  with both inputs and targets fields.
  Param: tf.data.Dataset
  Return: text-to-text format
  """
  prefix = 'LINK CODE TO COMMENT: ' #@param {type : "string"}
  def to_inputs_and_targets(ex):
    x_input = tf.strings.strip(prefix + ex['input'])
    y_label = tf.strings.strip(ex['output']) 
    inputs = tf.strings.join([x_input], separator=' ')
    class_label = tf.strings.join([y_label], separator=' ')
    return {'inputs': inputs, 'targets': class_label}
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  from tensorflow_datasets.core.utils.type_utils import Shape

def preprocessing_classification(ds):
  """
  Preprocess function to convert the tf.data.Dataset into a text-to-text format,
  with both inputs and targets fields.
  Param: tf.data.Dataset
  Return: text-to-text format
  """
  prefix = 'CODE COMMENT CLASSIFICATION: ' #@param {type : "string"}
  def to_inputs_and_targets(ex):
    x_input = tf.strings.strip(prefix + ex['input'])
    y_label = tf.strings.strip(ex['output']) 
    inputs = tf.strings.join([x_input], separator=' ')
    class_label = tf.strings.join([y_label], separator=' ')
    return {'inputs': inputs, 'targets': class_label}
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

### A few examples

In [ ]:
print("A few preprocessed train examples...")
sample = tfds.as_numpy(preprocessing_linking(load_dataset_linking("train").take(5)))
for ex in sample:
  print(ex)

# Creating Task and Mixture

In [ ]:
DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=load_vocabulary(), add_eos=True, required=False),
    "targets": Feature(
        vocabulary=load_vocabulary(), add_eos=True)
    }

TASK_NAME_LINKING = "linking" #@param{ type : "string"}

# TASK
t5.data.TaskRegistry.remove(TASK_NAME_LINKING)
t5.data.TaskRegistry.add(
    TASK_NAME_LINKING,
    # Function which returns a tf.data.Dataset
    dataset_fn=load_dataset_linking,
    splits=["train","validation"],
    # List of functions that preprocess the input tf.data.Dataset
    text_preprocessor=[preprocessing_linking],
    # Accuracy is used as evaluation metric
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, helps for mixing and auto-caching
    # num_input_examples=num_input_examples,
    output_features = DEFAULT_OUTPUT_FEATURES
)

TASK_NAME_CLASSIFICATION = "classification" #@param{ type : "string"}

t5.data.TaskRegistry.remove(TASK_NAME_CLASSIFICATION)
t5.data.TaskRegistry.add(
    TASK_NAME_CLASSIFICATION,
    # Function which returns a tf.data.Dataset
    dataset_fn=load_dataset_classification,
    splits=["train","validation"],
    # List of functions that preprocess the input tf.data.Dataset
    text_preprocessor=[preprocessing_classification],
    # Accuracy is used as evaluation metric
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, helps for mixing and auto-caching
    # num_input_examples=num_input_examples,
    output_features = DEFAULT_OUTPUT_FEATURES
)

MIXTURE_NAME = "task" #@param{ type : "string"}

# MIXTURE
t5.data.MixtureRegistry.remove(MIXTURE_NAME)
t5.data.MixtureRegistry.add(
    MIXTURE_NAME,
    # List of tasks
    [TASK_NAME_LINKING, TASK_NAME_CLASSIFICATION],
    default_rate=1.0
)

### A few examples

In [ ]:
finetuning_task = t5.data.TaskRegistry.get(TASK_NAME_CLASSIFICATION)
ds = finetuning_task.get_dataset(split="train", sequence_length={"inputs": 1250, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

# Creating Model

In [ ]:
scheduler = "constant" #@param ["polynomial", "constant", "isr", "slanted"]

In [ ]:
# Storage paths
FINETUNE_MODEL_DIR =  "gs://cs-summarizer/linking&classification/models/with-pretraining-line-level/Best-Performing-Model"
PRETRAIN_MODEL_DIR='gs://cs-summarizer/linking&classification/models/pretrained-model'
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string ('f', '', 'kernel')

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

# Learning rate properties
starter_learning_rate = 0.01 #@param {type : "number"}
end_learning_rate = 0.001 #@param {type : "number"}
decay_steps = 10000 #@param {type : "integer"}

learning_rate_fn = PolynomialDecay(
     starter_learning_rate,
     decay_steps,
     end_learning_rate,
     power=0.5)

In [ ]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular, truncated_rsqrt
from t5 import models

# Learning rate schedule fn
if scheduler == 'polynomial':
  learning_rate_scheduler = learning_rate_fn
elif scheduler == 'isr':
  learning_rate_scheduler = truncated_rsqrt
elif scheduler == 'slanted':
  learning_rate_scheduler = slanted_triangular
else: 
  learning_rate_scheduler = 0.001

print(learning_rate_scheduler)

# Model properties
MODEL_SIZE = "small" 
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 32, 100),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]


# Mesh Tensorflow Transformer
model = t5.models.MtfModel(
    model_dir=FINETUNE_MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size, 
    sequence_length={"inputs": 1250, "targets": 256}, 
    # pick the correct scheduler, according to the model you want to train
    learning_rate_schedule = learning_rate_scheduler, 
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max,
    iterations_per_loop=100,
)

# Learning Scheduler

In [ ]:
#Upload the operative_config.gin according to the scheduler that has been picked
LOCAL_GIN_PATH = "/content/operative_config.gin"
with gin.unlock_config():
    gin.parse_config_file(LOCAL_GIN_PATH)

# Finetuning the Model

In [ ]:
TRAIN_STEPS =  75000#@param {type: "integer"}

# Start finetuning
#1) If no pre-training, then method model.train

model.finetune(mixture_or_task_name=MIXTURE_NAME,
               finetune_steps=TRAIN_STEPS, 
               pretrained_model_dir=PRETRAIN_MODEL_DIR)

# model.train(mixture_or_task_name=MIXTURE_NAME, steps=TRAIN_STEPS)

model.eval(
    mixture_or_task_name=MIXTURE_NAME,
    checkpoint_steps=-1
)